# UCI Epileptic Seizure Recognition Data Set

Link to the dataset: https://archive.ics.uci.edu/ml/datasets/Epileptic+Seizure+Recognition

In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

##  Hyperparamters

In [3]:
batch_size = 64
test_split = .2
shuffle_dataset = True
random_seed = 42
epochs = 100
learning_rate = 1e-3

## Dataset

In [15]:
class UCIEpilepsy(Dataset):
    def __init__(self, data_path):
        self.data = pd.read_csv(data_path).iloc[:, 1:].to_numpy().astype(np.single)
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):        
        data = torch.from_numpy(self.data[idx,2:178])
        label = torch.from_numpy(self.data[idx,178:])
        return data, label

dataset = UCIEpilepsy(data_path='data/data.csv')

dataset_size = len(dataset)

indices = list(range(dataset_size))
split = int(np.floor(test_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
test_indices, train_indices = indices[:split], indices[split:]

train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
test_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

## Neural Network

In [20]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(176, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 5)
        )
        
    def forward(self, x):
        return self.linear_relu_stack(x)

## Training Loop

In [21]:
def train_loop(X,y, model, loss_fn, optimizer):
    size = len(X)
    # for batch, (X, y) in enumerate(dataloader, 0):
    outputs = model(X)

    loss = loss_fn(outputs, y.squeeze(1).long()-1)


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # if batch % 10 == 0:
    # loss, current = loss.item(), batch * len(X)
    print(f"loss: {loss:>7f}")

## Test Loop

In [22]:
def test_loop(X, y, model, criterion):
    size = len(X)
    num_correct = 0
    num_samples = 0
    
    with torch.no_grad():
        #for X, y in dataloader:
        outputs = model(X)
        _, predictions = outputs.max(1)
        num_correct += (predictions == y).sum()
        num_samples += predictions.size(0)
        
    print(f"Got {num_correct} / {num_samples} with {float(num_correct) / float(num_samples) * 100:.2f}")

## Execution

In [23]:
model = NeuralNetwork()
X, y = next(iter(train_dataloader))
textX, testy = next(iter(test_dataloader))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)


for epoch in range(epochs):
    print(f"Epoch [{epoch+1}/{epochs}]")
    train_loop(X, y, model, criterion, optimizer)
    test_loop(X, y, model, criterion)

Epoch [1/100]
loss: 18.408686
Got 9 / 64 with 14.06
Epoch [2/100]
loss: 13.081454
Got 15 / 64 with 23.44
Epoch [3/100]
loss: 13.784674
Got 345 / 64 with 539.06
Epoch [4/100]
loss: 8.900457
Got 735 / 64 with 1148.44
Epoch [5/100]
loss: 9.482249
Got 582 / 64 with 909.38
Epoch [6/100]
loss: 6.394184
Got 543 / 64 with 848.44
Epoch [7/100]
loss: 4.073208
Got 486 / 64 with 759.38
Epoch [8/100]
loss: 2.911988
Got 495 / 64 with 773.44
Epoch [9/100]
loss: 2.571485
Got 432 / 64 with 675.00
Epoch [10/100]
loss: 1.705727
Got 414 / 64 with 646.88
Epoch [11/100]
loss: 1.073463
Got 444 / 64 with 693.75
Epoch [12/100]
loss: 0.786391
Got 510 / 64 with 796.88
Epoch [13/100]
loss: 0.661567
Got 546 / 64 with 853.12
Epoch [14/100]
loss: 0.446909
Got 567 / 64 with 885.94
Epoch [15/100]
loss: 0.332226
Got 585 / 64 with 914.06
Epoch [16/100]
loss: 0.268042
Got 597 / 64 with 932.81
Epoch [17/100]
loss: 0.234971
Got 594 / 64 with 928.12
Epoch [18/100]
loss: 0.207456
Got 591 / 64 with 923.44
Epoch [19/100]
loss: